# Dataset Download

In [1]:
!sh ../download_TextDataset.sh

mkdir: cannot create directory ‘dataset/’: File exists
--2024-08-24 06:47:17--  https://huggingface.co/datasets/Mireu-Lab/Korean-jeju-dialect-speech/resolve/main/Processing/Text/csv/processing_utteranceList.csv
Resolving huggingface.co (huggingface.co)... 13.225.131.93, 13.225.131.6, 13.225.131.94, ...
Connecting to huggingface.co (huggingface.co)|13.225.131.93|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/97/a6/97a64cdb19efc35b94db6d9f1b5a01a60a88784965e2a9683966f50d01677271/5791ce2a3d2494ac94ce8e4214a248d3325ae09d9e91eecb68d4d3140c0261b8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27processing_utteranceList.csv%3B+filename%3D%22processing_utteranceList.csv%22%3B&response-content-type=text%2Fcsv&Expires=1724741238&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDc0MTIzOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzk3L2E2L

# Python Lib Install

In [2]:
!pip3 install pandas


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip3 install tokenizers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip3 install transformers


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


# BERT Word Embedding

## Dataset Load

In [5]:
import pandas as pd

sentenceDataFrame = pd.read_csv("dataset/utteranceList.csv")

In [6]:
sentenceDataFrame

,standard_form,dialect_form
0,언니 만나서 반가워 아 오늘 제주 방언 에이 아이 데이터,언니 만낭 반가워 아 오늘 제주 방언 에이 아이 데이터
1,어 그거 대화 주제는 오늘 대화 주제는,어 그거 대화 주제는 오늘 대화 주제는
2,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예요,어 명절 설 명절 추석 명절 요로케 나누어서 해볼 거예
3,자 그러면 이제부터 얘기해 봐요,자 그믄 이제부터 얘기해 보게예
4,어 그러면 언니네 설 명절 때 음식 어떻게 해?,어 그믄 언니네 설 명절 때 음식 어떵 해?
...,...,...
292044,마무리 해서 저녁에 잠깐 나갈게.,마무리 해그넹 저녁에 잠깐 나가크라.
292045,어 그렇게 해라,어 겅 하라
292046,카페나 가자.,카페나 가게.
292047,난 차 없으니까 우리 집 못가,난 차 어시난 우리 집 못가


# Tokenizer Train

In [7]:
from tokenizers import BertWordPieceTokenizer 

vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

In [8]:
# BertWordPieceTokenizer 선언
# None 자리는 원래 vocabulary 파일 경로가 들어가는 자리입니다.
# 하지만 저희는 새로운 vocabulary를 만들것이기 때문에 None으로 만듭니다.
# 그 외에는 자유롭게 변경하셔도 되는 부분입니다.
tokenizer = BertWordPieceTokenizer(
    None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##",
)

In [9]:
# 훈련하기
    # .train을 쓰면 파일 경로를 주어서 학습할 수 있고
    # .train_from_iterator를 쓰면은 List[str] 형태를 주어 학습할 수 있습니다.
    # special token과 vocab size를 정하고 학습합니다.
tokenizer.train_from_iterator(
    sentenceDataFrame["standard_form"].to_list(),
    vocab_size=36000,
    min_frequency=2,
    show_progress=True,
    special_tokens = ["[PAD]", "[CLS]", "[UNK]", "[SEP]", "[MASK]"],
    wordpieces_prefix="##",
)

## Tokenizer Train Save

In [10]:
import numpy as np

vocab_dir = 'standard_form.txt'

# tokenizer에서 vocab 가져오기
vocab = tokenizer.get_vocab()

# index 번호에 맞게 정렬
vocabulary = [[v, k] for k, v in vocab.items()]
vocabulary = sorted(vocabulary)
vocabulary = list(np.array(vocabulary)[:, 1])

# vocabulary 저장
with open(vocab_dir, 'w+') as lf:
    lf.write('\n'.join(vocabulary))

# Predict Tokenizer

In [11]:
from transformers import BertTokenizer

bert_tokenizer = BertTokenizer(vocab_dir, do_basic_tokenize=False)
bert_tokenizer = BertTokenizer.from_pretrained(vocab_dir, do_basic_tokenize=False)

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:2165: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be 

## Done!

In [12]:
index = 3

standard_form = sentenceDataFrame["standard_form"].to_list()
print(standard_form[index])
print(bert_tokenizer.tokenize(standard_form[index]))
print(bert_tokenizer.encode(standard_form[index]))

자 그러면 이제부터 얘기해 봐요
['자', '그러면', '이제부터', '얘기해', '봐요']
[1, 712, 2073, 10778, 3065, 5545, 3]
